In [15]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np

In [16]:
def parse_dates(date_series):
    for fmt in ('%Y-%m-%d', '%d-%m-%Y', '%d/%m/%Y', '%Y/%m/%d'):
        try:
            return pd.to_datetime(date_series, format=fmt)
        except ValueError:
            continue
    # fallback: let pandas infer
    return pd.to_datetime(date_series, dayfirst=True)

In [17]:
def df_to_gdf(df):
    df["geometry"] = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
    return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

In [18]:
def landsat_map_station(stations_df, landsat_df):
    stations_gdf = df_to_gdf(stations_df)
    landsat_gdf = df_to_gdf(landsat_df)
    
    stations_m = stations_gdf.to_crs(epsg=3857)
    landsat_m  = landsat_gdf.to_crs(epsg=3857)
    
    landsat_to_station = gpd.sjoin_nearest(
        landsat_m, # LEFT: each landsat point
        stations_m,  # RIGHT: station candidates
        how="left",
        distance_col="dist_m"
    )
    
    landsat_to_station["dist_km"] = landsat_to_station["dist_m"] / 1000
    landsat_to_station = landsat_to_station.rename(columns= {'Latitude_left': 'Latitude', 'Longitude_left': 'Longitude', 'Latitude_right': 'Latitude_stations', 'Longitude_right': 'Longitude_stations'})
    
    landsat_to_station['Sample Date'] = parse_dates(landsat_to_station['Sample Date'])
    
    return landsat_to_station

# landsat_to_station.head()

In [19]:
# landsat_to_station.to_csv('data/landsat_to_station.csv', index=False)

## Adding hydrochem properties to the station data
This is the dataset with the date and the STAT_ID

In [20]:
def merge_all(glorich_df, landsat_to_station, name):

    glorich_with_date = glorich_df.dropna(subset=['date'])
    landsat_sorted  = landsat_to_station.sort_values('Sample Date').reset_index(drop=True)
    glorich_sorted  = glorich_with_date.sort_values('date').reset_index(drop=True)
    
    # Nearest-date join, matched within the same station ───────────────────
    #   • direction='nearest'  → picks the glorich row whose date is closest
    #     (before OR after) to the landsat observation date
    #   • by='STAT_ID'         → only considers glorich rows for the same station
    #   • how='left' is implicit in merge_asof; stations with no glorich data
    #     will get NaN for all glorich columns (your case 3) ───────────────────
    
    merged = pd.merge_asof(
        landsat_sorted,
        glorich_sorted,
        left_on='Sample Date',
        right_on='date',
        by='STAT_ID',
        direction='nearest',
        suffixes=('_landsat', '_glorich')
    )
    merged['date_diff_days'] = (
        merged['Sample Date'] - merged['date']
    ).dt.days.abs()
    merged.drop(columns=['index_right', 'Latitude_stations', 'Longitude_stations'], inplace=True)
    merged = merged.rename(columns={'Latitude_landsat': 'Latitude', 'Longitude_landsat': 'Longitude'})
    
    merged = (
        merged
        .sort_values('SpecCond25C_reliability', ascending=False)
        .drop_duplicates(subset=['Latitude', 'Longitude', 'Sample Date'], keep='first')
    )
    landsat_ids = set(landsat_sorted['STAT_ID'].unique())
    glorich_ids = set(glorich_with_date['STAT_ID'].unique())
    
    print("Landsat STAT_IDs:  ", len(landsat_ids))
    print("Glorich STAT_IDs:  ", len(glorich_ids))
    print("Overlapping IDs:   ", len(landsat_ids & glorich_ids))
    print("Merged dataset length:", len(merged))
    
    merged.to_csv(f'data/{name}_ALL.csv', index=False)

    return merged

## Main coding part

In [21]:
# name = specify training, validation, testing
name = 'training'
stations_df = pd.read_csv('data/stations_with_conditions.csv')
glorich_df = pd.read_csv('data/final_imputed_hydrochem.csv')
glorich_df['date'] = pd.to_datetime(glorich_df['date'], format='%Y-%m-%d')

# just change this to validation / testing set
landsat_df = pd.read_csv('data/train+landsat.csv')

In [22]:
stations_df.head()

,STAT_ID,Latitude,Longitude,geometry,sc,ss,su,mt,va,vb,...,pi,GLC_Artificial,GLC_Managed,GLC_Water,GLC_Aquatic_Veg,GLC_PERC_COV,Popdens_00,Soil_pH,SOC,Soil_wetness
0,400001,-32.31,18.33,POINT (18.33 -32.31),0.0,0.38,0.23,0.0,0.0,0.0,...,0.0,0.0,0.25,0.005508,0.0,100.0,4.75,6.7,3.08,34.46
1,400002,-32.31,18.34,POINT (18.34 -32.31),0.0,0.38,0.23,0.0,0.0,0.0,...,0.0,0.0,0.25,0.005508,0.0,100.0,4.75,6.7,3.08,34.46
2,400003,-32.31,18.34,POINT (18.34 -32.31),0.0,0.38,0.23,0.0,0.0,0.0,...,0.0,0.0,0.25,0.005511,0.0,100.0,4.70,6.7,3.08,34.46
3,400004,-32.31,18.35,POINT (18.35 -32.31),0.0,0.38,0.23,0.0,0.0,0.0,...,0.0,0.0,0.25,0.005511,0.0,100.0,4.70,6.7,3.08,34.46
4,400005,-32.34,18.42,POINT (18.42 -32.34),0.0,0.38,0.22,0.0,0.0,0.0,...,0.0,0.0,0.25,0.002830,0.0,100.0,4.46,6.7,3.12,34.46


In [23]:
landsat_to_station = landsat_map_station(stations_df, landsat_df)

In [24]:
merged = merge_all(glorich_df, landsat_to_station, name=name)

Landsat STAT_IDs:   160
Glorich STAT_IDs:   876
Overlapping IDs:    134
Merged dataset length: 9319


In [25]:
merged[merged['Cl'].isna()]

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus,pet,_merge_terra,nir,green,...,SO4,SpecCond25C,pH,Alkalinity_reliability,Cl_reliability,DIP_reliability,SO4_reliability,SpecCond25C_reliability,pH_reliability,date_diff_days
12,-28.14194,26.41778,2011-01-04,94.718,227.9,132.0,93.8,both,14391.0,9840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,-31.10428,29.39972,2011-01-18,52.984,129.0,10.0,134.1,both,13263.8,11149.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,-34.03194,22.05333,2011-01-19,17.640,266.5,10.0,157.5,both,9058.0,7859.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,-34.03972,22.13333,2011-01-19,11.201,167.4,23.0,156.0,both,13491.5,9266.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,-34.02861,22.22222,2011-01-20,208.236,238.8,10.0,152.8,both,16624.5,9018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10174,-28.80128,31.95539,2015-12-10,62.000,236.0,20.0,169.6,both,18219.5,10042.5,...,NaN,NaN,7.190407,NaN,NaN,NaN,NaN,NaN,1.0,5.0
10180,-26.61944,27.98083,2015-12-14,128.368,712.0,20.0,190.0,both,16552.0,10084.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10182,-26.51028,28.35139,2015-12-15,165.900,747.0,137.0,183.8,both,14200.5,10094.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10213,-28.40444,30.01306,2015-12-22,69.200,292.0,20.0,181.5,both,14952.0,10745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
train_df = pd.read_csv('data/training_ALL.csv')

In [30]:
train_df.shape

(9319, 55)